<a href="https://colab.research.google.com/github/DrFrank25/Syndecan_4-Ag73/blob/main/gmx_MMPBSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Binding Free Energy Calculation (MM-PBSA + Entropy)**

In [ ]:
# ==========================================
# 🚀 Binding Free Energy Calculation (MM-PBSA + Entropy) in Colab
# ==========================================
# -- Import Libraris ----
import os shutil, subprocess, sys, json, mat
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:
# -----------------------------
# 0) Bootstrap micromamba + install deps (Colab-safe)
# -----------------------------
def run(cmd, **kw):
    print(f"\n$ {cmd}")
    cp = subprocess.run(cmd, shell=True, check=True, text=True, **kw)
    return cp

if not shutil.which("micromamba"):
    run("wget -qO- https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba")
    os.environ["MAMBA_ROOT_PREFIX"] = "/root/micromamba"
    run("./bin/micromamba shell init -s bash -p /root/micromamba")
    # Activate shell for this process
    os.environ["PATH"] = "/root/micromamba/bin:" + os.environ["PATH"]

# Create env and install
run("micromamba create -y -n gmx -c conda-forge gromacs gmx-mmpbsa python=3.11")
# Activate env for this process
os.environ["PATH"] = "/root/micromamba/envs/gmx/bin:" + os.environ["PATH"]

# Detect gmx_MMPBSA vs g_mmpbsa
MMPBSA_BIN = "gmx_MMPBSA" if shutil.which("gmx_MMPBSA") else ("g_mmpbsa" if shutil.which("g_mmpbsa") else None)
if MMPBSA_BIN is None:
    raise SystemExit("Could not find gmx_MMPBSA/g_mmpbsa after install. Abort.")


In [ ]:
# -----------------------------
# 1) Upload inputs
# -----------------------------
from google.colab import files
print("📂 Please upload (exact names):")
print("  Rep1: topol1.tpr, traj1.xtc, index1.ndx, mmpbsa1.in")
print("  Rep2: topol2.tpr, traj2.xtc, index2.ndx, mmpbsa2.in")
print("  Rep3: topol3.tpr, traj3.xtc, index3.ndx, mmpbsa3.in")
uploaded = files.upload()

In [ ]:
# -----------------------------
# 2) Config — group selections for QHA (covar/anaeig)
#    By default, '1 1' means: group #1 for fit and #1 for analysis (often 'Protein').
#    If your index has different IDs, change below (e.g., "echo 3 3 | ...").
# -----------------------------
COVAR_GROUPS = "1 1"  # fit group, analysis group
ANAEIG_GROUP = "1"    # analysis group
TEMP_K = 300          # used by gmx anaeig; typically read from traj, but kept for reference


In [ ]:
# -----------------------------
# 3) Helpers
# -----------------------------
def run_mmpbsa(rep_id, tpr, xtc, ndx, inp):
    out_dir = f"rep{rep_id}"
    os.makedirs(out_dir, exist_ok=True)
    out_file = f"{out_dir}/FINAL_RESULTS_MMPBSA.dat"
    cmd = f"{MMPBSA_BIN} -f {xtc} -s {tpr} -n {ndx} -i {inp} -o {out_file} -nogui"
    run(cmd)

    # Parse FINAL_RESULTS_MMPBSA.dat -> dict
    res = {}
    with open(out_file) as f:
        for line in f:
            if line.strip() and not line.startswith("#"):
                parts = line.split()
                if len(parts) >= 2:
                    key, val = parts[0], parts[1]
                    try:
                        res[key] = float(val)
                    except:
                        pass
    # Normalize expected keys if package labels differ
    # (Some builds label as 'DeltaE_vdw', 'DeltaE_el', etc.)
    key_map = {
        "DeltaE_vdw": "DeltaE_vdw",
        "E_vdw": "DeltaE_vdw",
        "DeltaE_ele": "DeltaE_ele",
        "E_ele": "DeltaE_ele",
        "DeltaG_pol": "DeltaG_pol",
        "G_polar": "DeltaG_pol",
        "DeltaG_np": "DeltaG_np",
        "G_nonpolar": "DeltaG_np",
        "DeltaTOTAL": "DeltaTOTAL",
        "TOTAL": "DeltaTOTAL"
    }
    normalized = {}
    for k, v in res.items():
        normalized[key_map.get(k, k)] = v

    # Compute ΔH from components if not present
    if "DeltaTOTAL" not in normalized:
        normalized["DeltaTOTAL"] = sum([
            normalized.get("DeltaE_vdw", 0.0),
            normalized.get("DeltaE_ele", 0.0),
            normalized.get("DeltaG_pol", 0.0),
            normalized.get("DeltaG_np", 0.0),
        ])
    return normalized

def run_entropy_qha(rep_id, tpr, xtc, ndx):
    """Compute -TΔS via QHA using gmx covar + gmx anaeig.
       Output: value (kJ/mol)."""
    out_dir = f"rep{rep_id}"
    # Covariance (eigenvalues/eigenvectors)
    cmd_covar = f"echo {COVAR_GROUPS} | gmx covar -s {tpr} -f {xtc} -n {ndx} -o {out_dir}/eigenval.xvg -v {out_dir}/eigenvec.trr -quiet"
    run(cmd_covar)
    # Entropy from eigenmodes
    cmd_ana = f"echo {ANAEIG_GROUP} | gmx anaeig -v {out_dir}/eigenvec.trr -entropy {out_dir}/entropy.xvg -quiet"
    run(cmd_ana)

    # Parse entropy.xvg: typically last numeric column per line; take the final line
    tds = None
    with open(f"{out_dir}/entropy.xvg") as f:
        for line in f:
            if line.startswith(("#", "@")) or not line.strip():
                continue
            parts = line.split()
            try:
                val = float(parts[-1])
                tds = val  # overwrite until last line
            except:
                pass
    if tds is None:
        raise RuntimeError(f"Could not parse -TΔS from rep{rep_id}/entropy.xvg")
    return tds  # already in kJ/mol


In [ ]:
# -----------------------------
# 4) Run pipeline for 3 replicates
# -----------------------------
reps = {
    1: {"tpr": "topol1.tpr", "xtc": "traj1.xtc", "ndx": "index1.ndx", "inp": "mmpbsa1.in"},
    2: {"tpr": "topol2.tpr", "xtc": "traj2.xtc", "ndx": "index2.ndx", "inp": "mmpbsa2.in"},
    3: {"tpr": "topol3.tpr", "xtc": "traj3.xtc", "ndx": "index3.ndx", "inp": "mmpbsa3.in"},
}

rows = []
for i, spec in reps.items():
    print(f"\n===== Running replicate {i} =====")
    mm = run_mmpbsa(i, spec["tpr"], spec["xtc"], spec["ndx"], spec["inp"])
    tds = run_entropy_qha(i, spec["tpr"], spec["xtc"], spec["ndx"])
    row = {
        "Replicate": f"Rep{i}",
        "ΔE_vdw": mm.get("DeltaE_vdw", float("nan")),
        "ΔE_ele": mm.get("DeltaE_ele", float("nan")),
        "ΔG_pol": mm.get("DeltaG_pol", float("nan")),
        "ΔG_np":  mm.get("DeltaG_np", float("nan")),
        "ΔH":     (mm.get("DeltaE_vdw",0)+mm.get("DeltaE_ele",0)+mm.get("DeltaG_pol",0)+mm.get("DeltaG_np",0)),
        "-TΔS":   tds,
    }
    row["ΔG_total"] = row["ΔH"] + row["-TΔS"]
    rows.append(row)

df = pd.DataFrame(rows).set_index("Replicate")

# Stats
mean = df.mean(numeric_only=True)
std  = df.std(numeric_only=True)

df_stats = pd.concat([df, pd.DataFrame([mean, std], index=["Mean","Std"])], axis=0)
display(df_stats)

# Save CSV
out_csv = "Binding_Energy_3Replicates_withEntropy.csv"
df_stats.to_csv(out_csv, float_format="%.4f")
print(f"✅ Saved: {out_csv}")

In [ ]:
# -----------------------------
# 5) Visualization — bar charts
# -----------------------------
# (a) Per-replicate decomposition
ax = df[["ΔE_vdw","ΔE_ele","ΔG_pol","ΔG_np","ΔH","-TΔS","ΔG_total"]].plot(kind="bar", figsize=(12,6), edgecolor="black")
ax.set_ylabel("Energy (kJ/mol)")
ax.set_title("Binding Free Energy Decomposition (with Entropy) — 3 Replicates")
ax.set_xticklabels(df.index, rotation=0)
plt.tight_layout()
plt.savefig("Energy_Decomposition_3Replicates_withEntropy.png", dpi=300)
plt.show()
print("📊 Saved: Energy_Decomposition_3Replicates_withEntropy.png")

# (b) Summary means ± std for key terms
summary_terms = ["ΔE_vdw","ΔE_ele","ΔG_pol","ΔG_np","ΔH","-TΔS","ΔG_total"]
means = mean[summary_terms]
errs  = std[summary_terms]

plt.figure(figsize=(10,6))
bars = plt.bar(summary_terms, means, yerr=errs, capsize=4, edgecolor="black")
plt.ylabel("Energy (kJ/mol)")
plt.title("Mean ± SD across Replicates — Energy Contributions")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("Energy_Decomposition_Mean_SD.png", dpi=1200)
plt.show()
print("📊 Saved: Energy_Decomposition_Mean_SD.png")

print("✅ Done.")